In [53]:
import requests
from pyquery import PyQuery
import pandas as pd


def get_buildings():
    response = requests.get(
        'https://cookieclicker.fandom.com/wiki/Buildings')
    pq = PyQuery(response.content)
    buildings = pd.read_html(pq('.wikitable').outer_html(), index_col=0)[
        0][['Base Cost', 'Base CpS']]
    return buildings


def get_upgrades(buildings):
    response = requests.get(
        'https://cookieclicker.fandom.com/wiki/Upgrades')
    pq = PyQuery(response.content)
    captions = pq('caption')
    upgrades = []
    for name in buildings.index:
        upgrades.append(pd.read_html(captions(':contains("%s")' %
                        name).parent().outer_html())[0][['Name', 'Unlock condition', 'Base price']])
    upgrades = pd.concat(upgrades, keys=buildings.index, axis=1)
    return upgrades


buildings = get_buildings()
upgrades = get_upgrades(buildings)
buildings.to_excel('buildings.xlsx')
upgrades.to_excel('upgrades.xlsx')
